In [1]:
import os
import time

import pandas as pd
import requests
from tqdm import tqdm

In [2]:
os.chdir("../../data/user_profiles")

In [3]:
staring_usernames = ["Fro116"]
num_iters = 10000

In [4]:
def get_friend_df(json):
    # turn the json response into a dataframe
    usernames = []
    for item in json["friends"]:
        usernames.append(item["username"])
    return pd.DataFrame({"friends": usernames})

In [5]:
def get_friends(usernames):
    friend_lists = []

    max_items_per_page = 300  # property of the API
    for username in tqdm(usernames):
        more_pages = True
        page = 1
        friends = []
        while more_pages:
            time.sleep(5)  # Jikan requires a minimum rate limit of 1 request per 4 secs
            response = requests.get(
                f"https://api.jikan.moe/v3/user/{username}/friends/{page}"
            )
            page_df = get_friend_df(response.json())
            friends.append(page_df)
            page += 1
            if len(page_df) < max_items_per_page:
                more_pages = False
        friend_df = pd.concat(friends)
        friend_df["username"] = username
        friend_lists.append(friend_df)
    friends = pd.concat(friend_lists, ignore_index=True)
    return friends

In [ ]:
usernames = set(staring_usernames)

friends_by_round = [pd.DataFrame()]
usernames_by_round = [set()]
for i in range(num_iters):
    new_usernames = usernames - usernames_by_round[i]
    if len(new_usernames) == 0:
        break
    new_friends = get_friends(new_usernames)
    friends_by_round.append(new_friends)

    usernames_by_round.append(usernames)
    usernames = set(new_friends["friends"]) | usernames_by_round[i] | new_usernames
    pd.Series(sorted(list(usernames))).to_csv("usernames.csv", header=False, index=False)

  1%|          | 59/4722 [06:34<8:47:17,  6.78s/it]

In [ ]:
usernames_by_round

In [ ]:
friends_by_round